### Get data from an API 

In [35]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

def get_weather_api():
    # Openweather API key
    api_key = os.getenv('openweather_api')
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"
    CITY = "Berlin"
    url = f'{BASE_URL}appid={api_key}&q={CITY}'
    
    response = requests.get(url).json()
    
    return response


weather = get_weather_api()
print(weather)

{'coord': {'lon': 13.4105, 'lat': 52.5244}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 291.19, 'feels_like': 291.09, 'temp_min': 289.16, 'temp_max': 293.14, 'pressure': 1008, 'humidity': 78}, 'visibility': 9000, 'wind': {'speed': 6.17, 'deg': 300}, 'clouds': {'all': 75}, 'dt': 1717592439, 'sys': {'type': 2, 'id': 2011538, 'country': 'DE', 'sunrise': 1717555589, 'sunset': 1717615418}, 'timezone': 7200, 'id': 2950159, 'name': 'Berlin', 'cod': 200}


### Connect the DB and Create a table to handle the Data

In [23]:
import psycopg2

# Database connection parameters
DATABASE_URL = os.getenv('weather_db_url')

# Connect to the database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

# Create tables if they don't exist
cur.execute('''
CREATE TABLE IF NOT EXISTS weather_data(
    date TIMESTAMP PRIMARY KEY,
    city VARCHAR,
    temp FLOAT,
    feels FLOAT,
    disc VARCHAR  
)
''')

conn.commit()

### Send data to DB 

In [33]:
import requests
import datetime 

def save_crypto_data(response):
   
    # Database connection parameters
    DATABASE_URL = os.getenv('weather_db_url')
    # Connect to the database
    conn = psycopg2.connect(DATABASE_URL)  
    # Connect to the PostgreSQL database
    cursor = conn.cursor()

    # Extract required fields
    dt = datetime.datetime.utcfromtimestamp(response.get('dt'))
    city = response.get('name')
    temp = round(response['main'].get('temp') - 273.15, 2)
    feels_like = round(response['main'].get('feels_like') - 273.15, 2)
    description = response['weather'][0].get('description')
     
    # Insert Data to the DB table   
    cursor.execute('''
        INSERT INTO weather_data (date, city, temp, feels, disc)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (date) DO NOTHING
    ''', (dt, city, temp, feels_like, description))

    conn.commit()
    cursor.close()
    conn.close()
    
    return "Data saved to Database successfully"


save_crypto_data(weather)

'Data saved to Database successfully'